In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image, ImageOps
from tqdm import tqdm
from typing import Optional
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
import keras
import keras.callbacks
import keras.optimizers
import keras.layers as kl

2023-06-17 22:19:13.417160: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-17 22:19:13.446657: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    dice_coef = numerator / (denominator + tf.keras.backend.epsilon())
    return 1 - dice_coef

In [4]:
DATA_PATH = '/app/data/imagenet_data/'
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
VALI_PATH = os.path.join(DATA_PATH, 'vali')
TEST_PATH = os.path.join(DATA_PATH, 'test')

TRAIN_COLOR_PATH = os.path.join(TRAIN_PATH, 'color')
VALI_COLOR_PATH = os.path.join(VALI_PATH, 'color')
TEST_COLOR_PATH = os.path.join(TEST_PATH, 'color')

TRAIN_GRAYSCALE_PATH = os.path.join(TRAIN_PATH, 'grayscale')
VALI_GRAYSCALE_PATH = os.path.join(VALI_PATH, 'grayscale')
TEST_GRAYSCALE_PATH = os.path.join(TEST_PATH, 'grayscale')

In [5]:
image_dimensions = (400,400)

In [6]:
def read_images(file_paths: list, *, resize_dimensions: Optional[tuple] = (400,400), show_progress: bool = True):

    images = []
    folder_contents = file_paths
    if show_progress:
        folder_contents = tqdm(file_paths)

    for filename in folder_contents:
        img = Image.open(filename)
        if resize_dimensions is not None:
            img = ImageOps.fit(img, resize_dimensions)
        img = np.asarray(img) / 255.0
        img = img.reshape((*img.shape, 1))

        images.append(tf.constant(img))
    
    return np.array(images)


In [7]:
def get_image_paths(folder: str, start: int, end: int):
    return [os.path.join(folder, file_name) for file_name in os.listdir(folder)[start:end]]

In [8]:
# train images
train_slice = 0, 100
train_color = read_images(get_image_paths(TRAIN_COLOR_PATH, *train_slice), resize_dimensions=image_dimensions)
train_gray = read_images(get_image_paths(TRAIN_GRAYSCALE_PATH, *train_slice), resize_dimensions=image_dimensions)

  0%|          | 0/100 [00:00<?, ?it/s]2023-06-17 22:19:14.788383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-17 22:19:14.798272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-17 22:19:14.798496: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/b

In [9]:
# vali images
vali_slice = 0, 300
vali_color = read_images(get_image_paths(VALI_COLOR_PATH, *vali_slice), resize_dimensions=image_dimensions)
vali_gray = read_images(get_image_paths(VALI_GRAYSCALE_PATH, *vali_slice), resize_dimensions=image_dimensions)

100%|██████████| 300/300 [00:00<00:00, 443.02it/s]


In [10]:
# test images
test_limit = 0, 100
test_color = read_images(get_image_paths(TEST_COLOR_PATH, *test_limit), resize_dimensions=image_dimensions)
test_gray = read_images(get_image_paths(TEST_GRAYSCALE_PATH, *test_limit), resize_dimensions=image_dimensions)

100%|██████████| 100/100 [00:00<00:00, 451.54it/s]


In [11]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [29]:
input_shape = (*image_dimensions, 3)

inputs = kl.Input(shape=input_shape)

x = kl.Conv2D(2, (3,3), activation='relu', padding='same', strides=1)(inputs)
x = kl.Conv2D(4, (3,3), activation='relu', padding='same')(x)
x = kl.Conv2D(6, (3,3), activation='relu', padding='same', strides=2)(x)
x = kl.Conv2D(8, (3,3), activation='relu', padding='same', strides=2)(x)
x = kl.UpSampling2D((2,2))(x)
x = kl.Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = kl.UpSampling2D((2,2))(x)
x = kl.Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = kl.UpSampling2D((2,2))(x)
outputs = kl.Conv2D(3, (3,3), activation='tanh', padding='same')(x)

In [30]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['mae'])

In [31]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 conv2d_35 (Conv2D)          (None, 400, 400, 2)       56        
                                                                 
 conv2d_36 (Conv2D)          (None, 400, 400, 4)       76        
                                                                 
 conv2d_37 (Conv2D)          (None, 200, 200, 6)       222       
                                                                 
 conv2d_38 (Conv2D)          (None, 100, 100, 8)       440       
                                                                 
 up_sampling2d_15 (UpSamplin  (None, 200, 200, 8)      0         
 g2D)                                                            
                                                           

In [ ]:
hist = model.fit(x=train_color, y=train_gray, batch_size=1, epochs=30, callbacks=[early_stopping], validation_data=(vali_color, vali_gray), verbose=1)

In [ ]:
df_hist = pd.DataFrame(hist.history)

In [ ]:
df_hist.plot()

In [ ]:
pred = model.predict(test_color)

In [ ]:
# display all 100 predictions in a plot
fig, axs = plt.subplots(10, 10, figsize=(20,20))
for i in range(10):
    for j in range(10):
        axs[i,j].imshow((pred[i*10+j]*255).astype(np.uint8))
        axs[i,j].axis('off')